<h1><center> 5c. Übung: Data Mining

<div class="alert alert-warning"><h4> Aufgabe 1: Klassifikation

Sie finden den Datensatz *hmeq_modeling* als csv-Datei im Git-Repo oder unter [www.creditriskanalytics.net](http://www.creditriskanalytics.net/).

Der Datensatz enthält Informationen zu den folgenden 13 Variablen:
- **BAD**: 1=Kreditausfall; 0=kein Kreditausfall
- **LOAN**: Höhe des Kreditantrags
- **MORTDUE**: fälliger Betrag für bestehende Hypotheken
- **VALUE**: Wert des aktuellen Eigentums
- **REASON**: DebtCon=Schuldenkonsolidierung; HomeImp=home improvement
- **JOB**: berufliche Kategorien
- **YOJ**: Jahre in der gegenwärtigen Beschäftigung
- **DEROG**: Anzahl negativer Meldungen
- **DELINQ**: Anzahl der in Verzug geratenen Kreditraten
- **CLAGE**: Alter des ältesten Kreditrahmens in Monaten
- **NINQ**: Anzahl der jüngsten Kreditanfragen
- **CLNO**: Anzahl der Kreditraten
- **DEBTINC**: Schulden-Einkommen-Verhältnis

Die Modellierungsaufgabe besteht darin, den Status der Zielvariable BAD auf Grundlage der restlichen Variaben zu prognostizieren. 

a) Laden Sie den Datensatz *hmeq_modeling* als Dataframe. Machen Sie sich einen Überblick über den Datensatz und zeigen Sie, dass keine fehlenden Werte enthalten sind.

<div class="alert alert-danger">Im Datensatz sind nominale Variablen <strong>one-hot kodiert</strong>. Haben wir beispielsweise die Variable "farben" mit den Ausprägungen blau, rot und grün, erstellen wir stattdessen drei binäre Variablen "blau", "rot" und "grün". Die drei Variablen haben die Ausprägung 1, wenn die Farbe zutrifft und 0 wenn nicht. 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("..\_Daten\hmeq_modeling.csv", index_col='ID')
df.head(5)

In [ ]:
if np.any(df.isna())==False:
    print("I confirm the data does not include missing values")
else:
    print("Actually, the data does include missing values")

b) Unterteilen Sie den Datensatz zufällig in einen Training- (70 %) und in einen Testdatensatz (30%).\
**Hinweis**: Die Targetvariable in dem Datensatz ist das Merkmal **BAD**.

In [ ]:
from sklearn.model_selection import train_test_split

y = df.pop("BAD")
X = df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

c) Stellen Sie sicher, dass das Verhältnis von 0 und 1 im Trainings- und Testdatensatz *fast identisch* zum Originaldatensatz ist.\
**Hinweis**: Sehen Sie sich hierfür das Argument *stratify* an.

In [ ]:
def class_ratio(y):
    return np.sum(y)/y.shape[0]

print("Class ratio in original data: ", class_ratio(y))
print("Class ratio in training data: ", class_ratio(y_train))
print("Class ratio in test data: ", class_ratio(y_test))

In [ ]:
# Argument stratify um sicherzustellen, dass die class distribution im Trainings-/Testdatensatz 
# so ähnlich wie möglich zu der class ratio im originaldatensatz ist
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=123)
print("Class ratio in original data: ", class_ratio(y))
print("Class ratio in training data: ", class_ratio(y_train))
print("Class ratio in test data: ", class_ratio(y_test))

d) Verwenden Sie die **sklearn** Bibliothek um eine logistische Regression, einen Decision Tree (default-Einstellungen, random_state=1234) und einen Random Forest (default-Einstellungen, random_state=1234) zu schätzen.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
learners = [LogisticRegression(penalty='none'), DecisionTreeClassifier(random_state=1234),
           RandomForestClassifier(random_state=1234)]

In [ ]:
for l in learners:
    l.fit(X_train, y_train)
# FutureWarning ignorieren

e) Bewerten Sie die Klassifikatoren auf Grundlage des Testdatensatzes mit Hilfe der ROC-Analyse. Erstellen Sie ein ROC-Diagramm, dass die Performance der drei Klassifikatoren darstellt. 

In [ ]:
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()

for l in learners:
    RocCurveDisplay.from_estimator(l, X_test, y_test, ax=ax)

ax.set_title('ROC curve of candidate classifiers')
ax.plot([0, 1], [0, 1], "r--", label='baseline');  # the random benchmark we need to add manually
plt.show()

<div class="alert alert-warning"><h4> Aufgabe 2: Keras

In dieser Aufgabe spezifizieren und trainieren wir ein Neuronales Netz für den HMEQ Datensatz.

a) Instanziieren Sie das sequenzielle Neuronales Netz **nn**.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
nn = keras.Sequential()

b) Legen Sie nun die Netzwerkarchitektur fest, in dem Sie nacheinander die Layer hinzufügen:
* Ein Dense Layer mit 10 Knoten und einer tanH Aktivierungsfunktion (Denken Sie an `input_shape`)
* Ein Dense Layer mit 5 Knoten und einer RELU Aktivierungsfunktion
* Ein Output Layer (Dense Layer) mit einem Knoten und einer sigmoid Aktivierungsfunktion

In [ ]:
nn.add(layers.Dense(units=10, activation='tanh', input_shape=(X_train.shape[1],)))
nn.add(layers.Dense(units=5, activation='relu'))
nn.add(layers.Dense(units=1, activation='sigmoid'))

c) Geben Sie die *summary* des Modells aus.

In [ ]:
nn.summary()

d) Verwenden Sie `keras.optimizers.SGD` als Optimizer mit einer Lernrate von 0.01 und weisen Sie das dem Objekt `opt` zu.

In [ ]:
opt = keras.optimizers.SGD(learning_rate=0.01)

e) Kompilieren Sie Ihr Modell (`nn.compile()`). Verwenden Sie hierfür den in Aufgabe e) instanziierten Optimizers, eine binary crossentropy und als Metrik die Accuracy.

In [ ]:
nn.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

f) Trainieren Sie nun Ihr Modell mit Ihrem in Aufgabe 1 erstellten Testdatensatz. Dabei soll die Anzahl an Epochen gleich 10 und der validation_split gleich 0.2 sein.

**Achtung**: Aktuell ist unser Datensatz als pandas Dataframe gespeichert. Bevor Sie die Teilaufgabe lösen, müssen Sie zunächst den Trainings- und Testdatensatz als numpy array mit dem Datentyp float abspeichern.<br>
Die Target-Variable muss ebenfalls als numpy array vorliegen.

In [ ]:
X_train, y_train = np.asarray(X_train).astype(np.float32), np.asarray(y_train)
X_test, y_test = np.asarray(X_test).astype(np.float32), np.asarray(y_test)

In [ ]:
history = nn.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=1) 

g) Was gibt Ihnen der nachfolgende Python-Code aus?

In [ ]:
score = nn.evaluate(X_test, y_test, verbose=0)

In [ ]:
help(nn.evaluate)

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

h) Lassen Sie sich mithilfe der in der Vorlesung definierten Funktion `show_history()` die Modellperformance ausgeben.

In [ ]:
import matplotlib.pyplot as plt

def show_history(story):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18.5, 10.5)
    ax1.plot(story.history['accuracy'])
    ax1.plot(story.history['val_accuracy'])
    ax1.set(xlabel='epoch', ylabel='accuracy')
    ax1.legend(['train_accuracy', 'test_accuracy'], loc='best')
    ax1.set_title('Accuracy evolution during NN training')
    
    ax2.plot(story.history['loss'])
    ax2.plot(story.history['val_loss'])
    ax2.set(xlabel='epoch', ylabel='loss')
    ax2.legend(['train_loss', 'test_loss'], loc='best')
    ax2.set_title('Loss evolution during NN training')
    plt.show()

show_history(history)